# Projet de traitement de données massive

## Partie 1

### Collecte de données

Pour Commencer le projet, j'ai décidé de prendre un dataset depuis kaggle directement puis par la suite mettre en place le telechargement automatique.

## Extract informations

In [7]:
import PIL.Image
import os
import json
from sklearn.cluster import KMeans
import numpy
from colorthief import ColorThief
from scipy.spatial import KDTree
from webcolors import hex_to_rgb
from webcolors import CSS3_HEX_TO_NAMES


def orientation(heigh, width):
    if heigh > width:
        return "Portrait"
    elif width > heigh:
        return "Paysage"
    else:
        return "Carre"

def extract_metadata(directory_name):
    directory = os.fsencode(directory_name)
    dataDict = {}
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        img = PIL.Image.open(directory_name+"/"+filename)
        exif_data = img._getexif()
        color_thief = ColorThief(directory_name+"/"+filename)
        dom = color_thief.get_color(quality=1)
        print(dom)
        print(dom[0])
        dataDict[filename] = {
            "Height": img.height, 
            "Width": img.width, 
            "Exif": str(exif_data), 
            "Couleurs dominantes" : color_thief.get_palette(color_count=5), 
            "Orientation": orientation(img.height, img.width),
            "Color name": convert_rgb_to_names(dom)
            }
        

    with open('json_data.json', 'w') as outfile:
        json.dump(dataDict, outfile)
    

def convert_rgb_to_names(rgb_tuple):
    
    # a dictionary of all the hex and their respective names in css3
    css3_db = CSS3_HEX_TO_NAMES
    names = []
    rgb_values = []
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(hex_to_rgb(color_hex))
    
    kdt_db = KDTree(rgb_values)
    distance, index = kdt_db.query(rgb_tuple)
    return f'closest match: {names[index]}'



extract_metadata("images_test")


(162, 112, 119)
162
(60, 28, 63)
60
(164, 115, 154)
164
